 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500 Index.

In [ ]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
import seaborn as sn

%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp500_history.csv`: Contains historical closing prices of the S&P 500 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [ ]:
# Reading whale returns
wr_data = Path('Resources/whale_returns.csv')
whale_returns = pd.read_csv(wr_data, index_col= 'Date', infer_datetime_format= True, parse_dates= True)

whale_returns.sort_index(inplace=True)
whale_returns.head()

In [ ]:
# Count nulls
whale_returns.isnull().sum()

In [ ]:
# Drop nulls
whale_returns = whale_returns.dropna()

whale_returns.head()

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [ ]:
# Reading algorithmic returns
ar_data = Path('Resources/algo_returns.csv')
algo_returns = pd.read_csv(ar_data, index_col='Date', infer_datetime_format=True, parse_dates=True)

algo_returns.sort_index(inplace=True)
algo_returns.head(10)

In [ ]:
# Count nulls
algo_returns.isnull().sum()

In [ ]:
# Drop nulls
algo_returns = algo_returns.dropna()

algo_returns.head()

## S&P 500 Returns

Read the S&P 500 historic closing prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P 500 Closing Prices
sp500_data = Path('Resources/sp500_history.csv')
sp500_history = pd.read_csv(sp500_data, index_col='Date', infer_datetime_format=True, parse_dates=True)

sp500_history.sort_index(inplace=True)
sp500_history.head()

In [ ]:
# Check Data Types
sp500_history.dtypes

In [ ]:
# Fix Data Types
sp500_history['Close'] = sp500_history['Close'].str.replace('$','')
sp500_history.head()


In [ ]:
sp500_history['Close'] = sp500_history['Close'].astype(float)
sp500_history.dtypes

In [ ]:
# Calculate Daily Returns
daily_returns = sp500_history.pct_change()

daily_returns.head()

In [ ]:
# Drop nulls
daily_returns = daily_returns.dropna()

daily_returns.head()


In [ ]:
# Rename `Close` Column to be specific to this portfolio.
daily_returns = daily_returns.rename(columns = {"Close":"S&P500"})
daily_returns.head()

## Combine Whale, Algorithmic, and S&P 500 Returns

In [ ]:
# Join Whale Returns, Algorithmic Returns, and the S&P 500 Returns into a single DataFrame with columns for each portfolio's returns.
all_df = pd.concat([whale_returns, algo_returns, daily_returns], axis="columns", join="inner")
all_df.head()

---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [ ]:
# Plot daily returns of all portfolios
all_df.plot(figsize=(20,10))

#### Calculate and Plot cumulative returns.

In [ ]:
# Calculate cumulative returns of all portfolios
cumulative_returns = (1 + all_df).cumprod()
# Plot cumulative returns
cumulative_returns.plot(figsize=(20,10))

---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

### Create a box plot for each portfolio


In [ ]:
# Box plot to visually show risk
all_df.plot.box(figsize=(20,10))

### Calculate Standard Deviations

In [ ]:
# Calculate the daily standard deviations of all portfolios
all_df.std()

### Determine which portfolios are riskier than the S&P 500

In [ ]:
# Calculate  the daily standard deviation of S&P 500
sp500_std = all_df["S&P500"].std()

# Determine which portfolios are riskier than the S&P 500
all_df.std() > sp500_std

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
annual_std = all_df.std() * np.sqrt(252)
annual_std

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for all portfolios using a 21-day window
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P 500
3. Choose one portfolio, then calculate and plot the 60-day rolling beta between it and the S&P 500

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [ ]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window
all_rolling_std = all_df.rolling(window=21).std()
# Plot the rolling standard deviation
all_rolling_std.plot(figsize=(20,10))


### Calculate and plot the correlation

In [ ]:
# Calculate the correlation
corr_df = all_df.corr()
# Display de correlation matrix
sn.heatmap(corr_df, annot=True)

In [ ]:
corr_df.style.background_gradient()

### Calculate and Plot Beta for a chosen portfolio and the S&P 500

In [ ]:
# Calculate covariance of a single portfolio
covariance_BH = all_df ["BERKSHIRE HATHAWAY INC"].rolling(window=60).cov(all_df["S&P500"])

# Calculate variance of S&P 500
variance_sp = all_df ["S&P500"].rolling(window=60).var()

# Computing beta
beta_BH = covariance_BH / variance_sp

# Plot beta trend
beta_BH.plot(figsize=(20,10), title="BERKSHIRE HATHWAY INC")

## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half life for each portfolio, using standard deviation (`std`) as the metric of interest.

In [ ]:
# Use `ewm` to calculate the rolling window
all_df.ewm(halflife=21).std().plot(figsize=(20,10))

---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Annualized Sharpe Ratios
sharpe_ratios = (all_df.mean() * 252) / (all_df.std()* np.sqrt(252))
sharpe_ratios

In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpe_ratios.plot.bar(figsize=(20,10))

### Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.

Write your answer here!

The algorithmic strategies 1 & 2 outperformed all three whale portfolios but only algorithmic strategies 1 outperformed the S&P500.

---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others
5. Include correlation analysis to determine which stocks (if any) are correlated

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [ ]:
# Reading data from 1st stock
appl_csv = Path('Resources/aapl_historical.csv')

appl_read = pd.read_csv(appl_csv, index_col= 'Trade DATE', infer_datetime_format=True, parse_dates=True)

appl_read.sort_index(inplace=True)
appl_read.head()

In [ ]:
# Reading data from 2nd stock
cost_csv = Path('Resources/cost_historical.csv')
cost_read = pd.read_csv(cost_csv, index_col= "Trade DATE", infer_datetime_format=True, parse_dates=True)

cost_read.sort_index(inplace=True)
cost_read.head()

In [ ]:
# Reading data from 3rd stock
goog_csv = Path('Resources/goog_historical.csv')
goog_read = pd.read_csv(goog_csv, index_col= "Trade DATE", infer_datetime_format=True, parse_dates=True)

goog_read.sort_index(inplace=True)
goog_read.head()

In [ ]:
# Combine all stocks in a single DataFrame
combined_df = pd.concat([appl_read, cost_read, goog_read], axis='columns', join="inner")

combined_df = combined_df.drop(columns=["Symbol"])
combined_df.head()

In [ ]:
# Reset Date index
combined_df.sort_index(inplace=True)
combined_df.head()

In [ ]:
# Reorganize portfolio data by having a column per symbol
combined_df.columns = ['APPL', 'COST', 'GOOG']

combined_df.head()

In [ ]:
# Calculate daily returns
daily_returns = combined_df.pct_change()

# Drop NAs
daily_returns = daily_returns.dropna()

# Display sample data
daily_returns.head()


## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Set weights
weights = [1/3, 1/3, 1/3]

# Calculate portfolio return
portfolio_df = daily_returns.dot(weights)

# Display sample data
portfolio_df.head()

## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Join your returns DataFrame to the original returns DataFrame
all_combined_df = pd.concat([combined_df, portfolio_df], axis="columns", join="inner")

columns = ['APPL', 'COST', 'GOOG', 'Daily Returns']
all_combined_df.columns = columns

all_combined_df

In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)
all_combined_df = all_combined_df.dropna()
all_combined_df

## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized `std`
all_combined_std = all_combined_df.std()
all_combined_std

### Calculate and plot rolling `std` with 21-day window

In [ ]:
# Calculate rolling standard deviation
rolling_std = daily_returns.rolling(window=21).std()

# Plot rolling standard deviation
rolling_std.plot(figsize=(20,10), title="Rolling STD 21-Day Window")



### Calculate and plot the correlation

In [ ]:
# Calculate and plot the correlation
correlation = daily_returns.corr()
correlation.plot(figsize=(20,10), title= "Correlation Chart")

### Calculate and Plot Rolling 60-day Beta for Your Portfolio compared to the S&P 500

In [ ]:
# Calculate and plot Beta
covariance_appl = all_combined_df["APPL"].rolling(window=60).cov(all_df["S&P500"])

variance_sp = all_df["S&P500"].rolling(window=60).var()

beta_appl = covariance_appl / variance_sp

# Plot beta trend
beta_appl.plot(figsize=(20,10), title= "Rolling 60-Day Beta: APPL to S&P500")

In [ ]:
# Calculate and plot Beta
covariance_cost = all_combined_df["COST"].rolling(window=60).cov(all_df["S&P500"])

variance_sp = all_df["S&P500"].rolling(window=60).var()

beta_cost = covariance_cost / variance_sp

# Plot beta trend
beta_cost.plot(figsize=(20,10), title= "Rolling 60-Day Beta: COST to S&P500")

In [ ]:
covariance_goog = all_combined_df["GOOG"].rolling(window=60).cov(all_df["S&P500"])

variance_sp = all_df["S&P500"].rolling(window=60).var()

beta_goog = covariance_goog / variance_sp

# Plot beta trend
beta_goog.plot(figsize=(20,10), title= "Rolling 60-Day Beta: GOOG to S&P500")

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Calculate Annualized Sharpe Ratios
sharpe_ratios = (daily_returns.mean() * 252) / (daily_returns.std()* np.sqrt(252))
sharpe_ratios

In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpe_ratios.plot.bar(figsize=(20,10))

### How does your portfolio do?

Write your answer here!

This portfolio does well with Costco (COST) signifcantly outperfoming other stocks within the portfolio.